In [1]:
import sys

sys.path.insert(0, '..')

#!{sys.executable} -m pip install folium
#!{sys.executable} -m pip install openapi_core


In [2]:
import morpheus.modflow.infrastructure.geometry.utils as utils
import morpheus.modflow.types.discretization.SpatialDiscretization as sd

polygon = sd.Polygon(
  type='Polygon',
  coordinates=[[
    (13.922514437551428, 50.964720483303836),
    (13.925250781947113, 50.965228748412386),
    (13.925036413951403, 50.96623732041704),
    (13.92222441026388, 50.96609040370362),
    (13.922514437551428, 50.964720483303836)
  ]]
)

rotation = sd.Rotation(20)
length_unit = sd.LengthUnit.meters()
x_coordinates = [0, 0.1, 0.21, 0.22, 0.23, 0.24, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
y_coordinates = [0, 0.2, 0.4, 0.6, 0.7, 0.75, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.88, 0.9, 0.95, 1]

grid = utils.grid_from_polygon(
  polygon=polygon,
  rotation=rotation,
  length_unit=length_unit,
  x_coordinates=x_coordinates,
  y_coordinates=y_coordinates
)
grid

Grid(x_coordinates=[0.0, 35.55136551768519, 74.6578675871389, 78.21300413890742, 81.76814069067593, 85.32327724244445, 88.87841379421297, 106.65409655305557, 142.20546207074077, 177.75682758842595, 213.30819310611113, 248.85955862379632, 284.41092414148153, 319.96228965916674, 355.5136551768519], y_coordinates=[0.0, 51.66873376723379, 103.33746753446758, 155.00620130170137, 180.84056818531826, 193.7577516271267, 206.67493506893516, 209.25837175729686, 211.84180844565853, 214.4252451340202, 217.0086818223819, 219.5921185107436, 222.17555519910528, 227.34242857582868, 232.50930195255205, 245.4264853943605, 258.34366883616894], origin=Point(coordinates=(13.922575153899507, 50.96461542216845), type='Point'), rotation=Rotation(value=20), length_unit=LengthUnit(unit=2))

In [3]:
import folium

m = folium.Map(tiles="cartodbpositron", crs="EPSG3857", zoom_start=12, location=[50.965, 13.922])
m.add_child(folium.GeoJson(polygon.as_geojson()))
m.fit_bounds(m.get_bounds())
m

In [4]:
from morpheus.modflow.infrastructure.geometry.utils import calculate_grid_geometry

grid_geometry = calculate_grid_geometry(grid)
m.add_child(folium.GeoJson(grid_geometry.as_geojson()))
m.fit_bounds(m.get_bounds())
m

In [5]:
from morpheus.modflow.infrastructure.geometry.utils import calculate_grid_cell_geometries

cell_geometries = calculate_grid_cell_geometries(grid)
for row in range(grid.ny()):
  for column in range(grid.nx()):
    folium.GeoJson(cell_geometries[row][column].as_geojson()).add_to(m)
m.fit_bounds(m.get_bounds())

m

In [6]:
from morpheus.modflow.infrastructure.geometry.utils import calculate_affected_cells_from_polygon, \
  calculate_affected_cells_geometries

affected_cells = calculate_affected_cells_from_polygon(polygon, grid)
affected_cells_geometries = calculate_affected_cells_geometries(affected_cells, grid)

m = folium.Map(tiles="cartodbpositron", crs="EPSG3857", zoom_start=12, location=[50.965, 13.922])
folium.GeoJson(polygon.as_geojson(), name="hello world").add_to(m)
folium.GeoJson(grid_geometry.as_geojson()).add_to(m)
folium.GeoJson(affected_cells_geometries.as_geojson()).add_to(m)
m.fit_bounds(m.get_bounds())
m